In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL.Image
import skimage 
from skimage.metrics import mean_squared_error as MSE
from skimage.metrics import peak_signal_noise_ratio as PSNR
from alive_progress import alive_bar
import time
from skimage.metrics import structural_similarity as SSIM
# import all functions needed
from core.MeanAndCovar import math_computation
from core.Transform import transform_matrix, transform_pixels
from core.Neigh import extract_neighborhood_info
from core.Estim import likelihood_estimation
device = torch.device("cuda")


# Read input data
original_image = cv2.imread('input_training_lowres/GT17.png')
trimap = cv2.imread('trimap/GT17.png')
ground_truth_trimap = cv2.imread('gt_training_lowres/GT17.png')

#avoid picture being blue
original_image = original_image[...,::-1]
trimap = trimap[...,::-1]
ground_truth_trimap = ground_truth_trimap[...,::-1]

print(trimap.shape)

ModuleNotFoundError: No module named 'torch'

In [ ]:
plt.imshow(original_image)
plt.axis('off')  # close axis
plt.show()

plt.imshow(trimap)
plt.axis('off')  # close axis
plt.show()

plt.imshow(ground_truth_trimap)
plt.axis('off')  # close axis
plt.show()

print(trimap.shape)

In [ ]:
sizeimg_2_dims = original_image.shape[:2]
sizetrimap = trimap.shape[:2]

if sizeimg_2_dims == sizetrimap:
    print("sizechecking: Successfull alignment of image and trimap!")
#else:
#    print("sizechecking: size not match, asking for new trimap!")
#    File, Path = uigetfile(
#        ('*.jpg;*.jpeg; *.png; *.gif;*.bmp;*.tiff',
#         'Image files (.jpg, .jpeg, .png, .gif, .bmp, .tiff'),
#        'Select a new trimap')
#    trimap = cv2.imread(Path + File)

if trimap.shape[2] != 1:
    trimap = cv2.cvtColor(trimap, cv2.COLOR_RGB2GRAY)

print(trimap.shape)

In [ ]:
# Convert input data to double ranged from 0 to 1
original_image = original_image.astype(np.double) / 255.0
ground_truth_trimap = ground_truth_trimap.astype(np.double) / 255.0
rows, cols, c = original_image.shape

trimap_show = np.stack([trimap, trimap, trimap], axis = 2)

plt.imshow(original_image)
plt.axis('off')  # close axis
plt.show()

plt.imshow(trimap_show)
plt.axis('off')  # close axis
plt.show()
# Define foreground, background, and unknown areas
fgmask = (trimap == 255)
bgmask = (trimap == 0)
unknown = (trimap == 128)
print(original_image)
print(ground_truth_trimap)
print(rows)
print(cols)
print(c)
print(fgmask)
print(bgmask)
print(unknown)



In [ ]:
# Initialize alpha matte (background: 0, foreground: 1, unknown area: NaN)
alpha = np.zeros([rows, cols])
alpha[fgmask] = 1
alpha[bgmask] = 0
alpha[unknown] = np.nan
alpha = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)

# Initialize F and B matrices
F = np.copy(original_image)
F[~fgmask] = np.nan
B = np.copy(original_image)
B[~bgmask] = np.nan

# Copy initial values to be used later
F_init = np.copy(F)
B_init = np.copy(B)
alpha_init = np.copy(alpha)
print(F_init)
print(B_init)
print(alpha_init)
# Set parameters
initial_side_length = 41
increment_for_side_length = 8
sigma_for_gsn = 4
remaining_unknown_pxls = []



In [ ]:
# First Loop: Iterate over pixels in the unknown area
for i in range(rows):
    for j in range(cols):
        if unknown[i, j]:
            # Initialize parameters
            side_length = initial_side_length
            increment = increment_for_side_length

            # Calculate the number of foreground and background pixels required
            num_fg_pxls = 0
            num_bg_pxls = 0
            min_fg_pxls = 200
            min_bg_pxls = 200
            max_side_length = 75

            # Iterate to find neighborhood information
            for num_iterations in range(max_side_length):
                if ((num_fg_pxls >= min_fg_pxls) and (num_bg_pxls >= min_bg_pxls)) or (side_length > max_side_length):
                    break

                # Get neighborhood information
                fg_neighb, bg_neighb, fg_weights, bg_weights, initial_alpha = extract_neighborhood_info(
                    F_init, B_init, alpha_init, i, j, side_length, sigma_for_gsn)

                # Calculate the number of foreground and background pixels in the neighborhood
                num_fg_pxls = (np.count_nonzero(~np.isnan(fg_neighb))) / 3
                num_bg_pxls = (np.count_nonzero(~np.isnan(bg_neighb))) / 3

                # Increment window size
                side_length = side_length + increment

            # Store remaining unknown pixels if information is insufficient
            if side_length > max_side_length:
                remaining_unknown_pxls.append([i, j])
                continue

            # Estimate alpha, F, B values for pixels with enough information
            fg_weights = transform_matrix(fg_weights, fg_neighb)
            bg_weights = transform_matrix(bg_weights, bg_neighb)
            pxls_fg = transform_pixels(fg_neighb, fg_neighb)
            pxls_bg = transform_pixels(bg_neighb, bg_neighb)
            F_covar, B_covar, B_mean, F_mean = math_computation(
                pxls_fg, pxls_bg, bg_weights, fg_weights)
            C = original_image[i, j, :]
            C = C.reshape((3, 1))
            max_iterations = 50
            min_likelihood = 1e-6
            sigma_C = 0.01
            F_mean = F_mean.T
            B_mean = B_mean.T
            F_element, B_element, alpha_element = likelihood_estimation(
                C, initial_alpha, F_mean, B_mean, F_covar, B_covar,
                sigma_C, max_iterations, min_likelihood)

            F_element[np.isnan(F_element)] = 0
            B_element[np.isnan(B_element)] = 0


            F[i, j, :] = F_element.T
            B[i, j, :] = B_element.T
            alpha[i, j, :] = alpha_element



In [ ]:
# Second Loop: Iterate over remaining unknown pixels
for k in range(len(remaining_unknown_pxls)):
    i = remaining_unknown_pxls[k][0]
    j = remaining_unknown_pxls[k][1]

    side_length = initial_side_length
    increment = increment_for_side_length
    num_fg_pxls = 0
    num_bg_pxls = 0
    min_fg_pxls = 200
    min_bg_pxls = 200
    max_side_length = 80

    # Get neighborhood information for remaining pixels
    for num_iterations in range(max_side_length):
        if ((num_fg_pxls >= min_fg_pxls) and (num_bg_pxls >= min_bg_pxls)) or (side_length > max_side_length):
            break

        fg_neighb, bg_neighb, fg_weights, bg_weights, initial_alpha = extract_neighborhood_info(
            F, B, alpha, i, j, side_length, sigma_for_gsn)
        num_fg_pxls = (np.sum(~np.isnan(fg_neighb))) / 3
        num_bg_pxls = (np.sum(~np.isnan(bg_neighb))) / 3
        side_length = side_length + increment

    fg_weights = transform_matrix(fg_weights, fg_neighb)
    bg_weights = transform_matrix(bg_weights, bg_neighb)
    pxls_fg = transform_pixels(fg_neighb, fg_neighb)
    pxls_bg = transform_pixels(bg_neighb, bg_neighb)

    # Handle remaining pixels with insufficient information
    if side_length >= max_side_length:
        if num_fg_pxls > num_bg_pxls:
            alpha[i, j, :] = 1
            F_covar, B_covar, B_mean, F_mean = math_computation(
                pxls_fg, pxls_bg, bg_weights, fg_weights)
            F[i, j, :] = F_mean
            continue
        elif num_fg_pxls < num_bg_pxls:
            alpha[i, j, :] = 0
            F_covar, B_covar, B_mean, F_mean = math_computation(
                pxls_fg, pxls_bg, bg_weights, fg_weights)
            B[i, j, :] = B_mean
            continue
    else:
        # Estimate alpha, F, B values for pixels with enough information
        F_covar, B_covar, B_mean, F_mean = math_computation(
            pxls_fg, pxls_bg, bg_weights, fg_weights)
        
        C = original_image[i, j, :]
        C = C.reshape((3, 1))
        max_iterations = 50
        min_likelihood = 1e-6
        sigma_C = 0.01
        F_mean = F_mean.T
        B_mean = B_mean.T
        F_element, B_element, alpha_element = likelihood_estimation(
            C, initial_alpha, F_mean, B_mean, F_covar, B_covar,
            sigma_C, max_iterations, min_likelihood)

        F_element[np.isnan(F_element)] = 0
        B_element[np.isnan(B_element)] = 0

        F[i, j, :] = F_element.T
        B[i, j, :] = B_element.T
        alpha[i, j, :] = alpha_element
alpha_float32 = alpha.astype(np.float32)
alpha_1 = cv2.cvtColor(alpha_float32, cv2.COLOR_BGR2GRAY)
ground_truth_trimap_1 = ground_truth_trimap.astype(np.float32)
ground_truth_trimap_1 = cv2.cvtColor(ground_truth_trimap_1, cv2.COLOR_BGR2GRAY)
psnr_value = PSNR(alpha_1, ground_truth_trimap_1)
mse_value = MSE(alpha_1, ground_truth_trimap_1)
ssim_value = SSIM(alpha_1, ground_truth_trimap_1)        

F[np.isnan(F)] = 0
B[np.isnan(B)] = 0

# Display results
trimap = trimap.astype('double')
print(trimap.shape)
print(trimap)
plt.imshow(trimap)
plt.axis('off')  # close axis
plt.show()


plt.imshow(alpha)
plt.axis('off')  # close axis
plt.show()


plt.imshow(ground_truth_trimap)
plt.axis('off')  # close axis
plt.show()


plt.imshow(original_image)
plt.axis('off')  # close axis
plt.show()


plt.imshow(F)
plt.axis('off')  # close axis
plt.show()


plt.imshow(B)
plt.axis('off')  # close axis
plt.show()

print(f'The PSNR of Bayesian matting is: {PSNR:.5f}')
print(f'The MSE of Bayesian matting is: {MSE:.5f}')
cv2.waitKey(0)
cv2.destroyAllWindows()